# Treinamento de Modelo para Extração de Dados de Recibos Fiscais

Este notebook demonstra o fluxo de trabalho completo para preparar dados e treinar um modelo de reconhecimento de entidades nomeadas (NER) para extrair informações de recibos fiscais, como:
- Tipo de documento
- CNPJ
- Chave de acesso
- Data de emissão
- Valor total
- Número do documento
- Série

O processo envolve as seguintes etapas:
1. Instalação das dependências necessárias
2. Extração dos textos dos recibos do arquivo dataset.dart
3. Preparação e limpeza dos dados
4. Criação de anotações para NER
5. Treinamento de um modelo BERT pré-treinado para português
6. Avaliação do modelo
7. Uso do modelo para extrair informações de novos recibos

## 1. Instalação das Dependências

In [1]:
# Instalar as dependências necessárias
!pip install numpy pandas scikit-learn torch transformers datasets matplotlib tqdm seqeval

## 2. Importações e Configurações Iniciais

In [2]:
import os
import json
import re
import random
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorForTokenClassification
)
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from seqeval.metrics import classification_report, f1_score
from typing import List, Dict, Any, Tuple


# Configurações
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"  # Modelo pré-treinado em português
OUTPUT_DIR = "receipt_ner_model"
MAX_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 10
SEED = 42

# Configurar as sementes para reprodutibilidade
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

/Users/matheus.oliveira/Documents/custom_model/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Preparação do Dataset

### 3.1 Definição das Entidades e Constantes

In [3]:
# Define as entidades que queremos extrair
ENTITIES = [
    "TIPO_DOCUMENTO",
    "CNPJ",
    "CHAVE_ACESSO",
    "DATA_EMISSAO",
    "VALOR_TOTAL",
    "NUMERO_DOCUMENTO",
    "SERIE"
]

# Defina as etiquetas para o modelo NER
LABELS = [
    "O",  # Outside (não é uma entidade)
    "B-TIPO_DOCUMENTO", "I-TIPO_DOCUMENTO",  # Tipo de documento (NFCe, NFe, SAT, CTe...)
    "B-CNPJ", "I-CNPJ",  # CNPJ
    "B-CHAVE_ACESSO", "I-CHAVE_ACESSO",  # Chave de acesso
    "B-DATA_EMISSAO", "I-DATA_EMISSAO",  # Data de emissão
    "B-VALOR_TOTAL", "I-VALOR_TOTAL",  # Valor total
    "B-NUMERO_DOCUMENTO", "I-NUMERO_DOCUMENTO",  # Número do documento
    "B-SERIE", "I-SERIE"  # Série do documento
]

# Mapeamento de ID para etiqueta e vice-versa
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}

# Mapeamento dos códigos de modelo de documento para seus tipos
DOCUMENT_TYPE_MAP = {
    '01': 'NF',
    '02': 'NFVC',
    '04': 'NFP',
    '06': 'NFCE',
    '07': 'NFST',
    '08': 'CTRC',
    '09': 'CTAC',
    '10': 'CA',
    '11': 'CTFC',
    '13': 'BPR',
    '14': 'BPA',
    '15': 'BPNB',
    '16': 'BPF',
    '17': 'DT',
    '18': 'RMD',
    '20': 'OCC',
    '21': 'NFSC',
    '22': 'NFST',
    '23': 'GNRE',
    '24': 'AC',
    '25': 'MC',
    '26': 'CTMC',
    '27': 'NFTFC',
    '28': 'NFCG',
    '29': 'NFCA',
    '30': 'BRP',
    '2D': 'CFECF',
    '2E': 'BPEC',
    '55': 'NFE',
    '57': 'CTE',
    '59': 'CF',
    '60': 'CFEECF',
    '65': 'NFCE',
    '67': 'CTE',
    '8B': 'CTCA',
}

### 3.2 Extração dos Textos do Dataset

In [4]:
# As the data has already been extracted and is available in dataset_prepared/,
# we'll skip the extraction from dataset.dart and load the prepared data directly.

print("Carregando dados já preparados...")

# Verificar se os arquivos necessários existem
import os

raw_texts_path = "dataset_prepared/raw_texts.txt"
annotations_path = "dataset_prepared/annotations.json"
extracted_data_path = "dataset_prepared/extracted_data.json"

texts = []
annotations = []

# Carregar os textos brutos
if os.path.exists(raw_texts_path):
    with open(raw_texts_path, 'r', encoding='utf-8') as f:
        content = f.read()
        texts = content.split("\n\n---SEPARATOR---\n\n")
        texts = [text.strip() for text in texts if text.strip()]
    print(f"Carregados {len(texts)} textos do arquivo {raw_texts_path}.")
    
    # Exibir uma amostra para verificação
    if texts:
        print("\nExemplo de texto carregado:")
        print("-" * 80)
        print(texts[0][:500] + "..." if len(texts[0]) > 500 else texts[0])
        print("-" * 80)
else:
    print(f"Arquivo {raw_texts_path} não encontrado.")

# Carregar as anotações
if os.path.exists(annotations_path):
    with open(annotations_path, 'r', encoding='utf-8') as f:
        annotations = json.load(f)
    print(f"Carregadas {len(annotations)} anotações do arquivo {annotations_path}.")
else:
    print(f"Arquivo {annotations_path} não encontrado.")

# Carregar os dados extraídos
if os.path.exists(extracted_data_path):
    with open(extracted_data_path, 'r', encoding='utf-8') as f:
        extracted_data = json.load(f)
    print(f"Carregados {len(extracted_data)} registros de dados extraídos do arquivo {extracted_data_path}.")
else:
    print(f"Arquivo {extracted_data_path} não encontrado.")

Carregando dados já preparados...
Carregados 1 textos do arquivo dataset_prepared/raw_texts.txt.

Exemplo de texto carregado:
--------------------------------------------------------------------------------
=== TEXTO 1 ===
COOP COOPERATIVA DE CUNSURY AGOSTO, 3045 • INHN - Tatuo - . CNPJ57.508.426/0007-63 IE687.134.677.115 IM 25.457 208748 CUPON FISCAL ELETRONICO - SAT #| COD | DESC | QTD IUNIVL UN RSI (UL TR R) * IUL ITEM R 001 13624 002 12083 0. 352% fa 003 7891999144983 KE CREN TRAD 1cda 11390 3.08, Desconto sobre iten 004 12347 PAO FRC 8550KG 005 7834900283° 8 48 45 8.41) Total bruto de itens OTRI is descontos sobre iten PT' 6.68 4.05 9, 49 • 30 .99 10.84 10.49 46:38 Ponte: e 0 o 8. ore 0, 09 VO...
--------------------------------------------------------------------------------
Carregadas 27 anotações do arquivo dataset_prepared/annotations.json.
Carregados 27 registros de dados extraídos do arquivo dataset_prepared/extracted_data.json.


### 3.3 Funções Auxiliares para Extração de Informações

In [5]:
def is_valid_access_key(key: str) -> bool:
    """
    Verifica se a chave de acesso é válida com base nos prefixos conhecidos.
    """
    # Lista de prefixos válidos para chaves de acesso
    valid_prefixes = [
        "11", "12", "13", "14", "15", "16", "17", "21", "22", "23", "24", "25", 
        "26", "27", "28", "29", "31", "32", "33", "35", "41", "42", "43", "50", 
        "51", "52", "53"
    ]
    
    # Verifica se a chave tem 44 dígitos e começa com um prefixo válido
    if len(key) == 44 and key[:2] in valid_prefixes:
        return True
    return False

def extract_date_info_from_key(key: str) -> tuple:
    """
    Extrai o ano e mês da chave de acesso.
    
    Args:
        key: Chave de acesso de 44 dígitos
        
    Returns:
        Tupla contendo (ano, mês) da chave
    """
    if len(key) != 44:
        return None, None
    
    # Ano está nas posições 2-4 (2 dígitos)
    year = key[2:4]
    # Mês está nas posições 4-6 (2 dígitos)
    month = key[4:6]
    
    # Converte para números, adiciona 2000 ao ano para ter formato de 4 dígitos
    try:
        year = int(year)
        month = int(month)
        if 0 < month <= 12:  # Valida o mês
            # Formata para 4 dígitos (assumindo anos 2000)
            full_year = 2000 + year
            return str(full_year), f"{month:02d}"
    except ValueError:
        pass
    
    return None, None

def extract_document_type_from_key(key: str) -> str:
    """
    Extrai o tipo de documento a partir da chave de acesso.
    
    Args:
        key: Chave de acesso de 44 dígitos
        
    Returns:
        String com o tipo de documento ou string vazia se não for reconhecido
    """
    if len(key) != 44:
        return ""
    
    # O tipo de documento está nas posições 20-22
    model_code = key[20:22]
    
    # Retorna o tipo de documento correspondente ou string vazia se não encontrado
    return DOCUMENT_TYPE_MAP.get(model_code, "")

def clean_currency_value(value: str) -> str:
    """
    Limpa e padroniza um valor monetário.
    
    Args:
        value: Valor monetário como string (ex: "114,54", "R$ 50.00", etc.)
        
    Returns:
        Valor padronizado como string decimal com ponto (ex: "114.54")
    """
    if not value:
        return ""
    
    # Remove qualquer caractere que não seja dígito, vírgula ou ponto
    value = re.sub(r'[^\d,.]', '', value)
    
    # Substitui vírgula por ponto para padronização decimal
    value = value.replace(',', '.')
    
    # Se houver mais de um ponto, mantém apenas o último (caso de milhares)
    if value.count('.') > 1:
        parts = value.split('.')
        last_part = parts[-1]
        rest = ''.join(parts[:-1]).replace('.', '')
        value = f"{rest}.{last_part}"
    
    return value

### 3.4 Extração de Anotações dos Textos

In [6]:
# Como as anotações já foram criadas anteriormente e estão disponíveis em dataset_prepared/annotations.json,
# vamos pular a etapa de criação de anotações.

print("Usando anotações previamente criadas...")

# Verificar se as anotações foram carregadas corretamente
if 'annotations' in locals() and annotations:
    print(f"Temos {len(annotations)} anotações disponíveis para uso.")
    
    # Exibir um exemplo para verificação
    if annotations:
        example = annotations[0]
        print("\nExemplo de anotação:")
        print("-" * 80)
        print(f"ID: {example.get('id', 'N/A')}")
        print(f"Texto (primeiros 100 caracteres): {example['text'][:100]}...")
        print("Anotações:")
        for key, value in example['data'].items():
            print(f"  - {key}: {value}")
        print("-" * 80)
else:
    print("Nenhuma anotação disponível. Verifique se o arquivo dataset_prepared/annotations.json existe e está correto.")

Usando anotações previamente criadas...
Temos 27 anotações disponíveis para uso.

Exemplo de anotação:
--------------------------------------------------------------------------------
ID: N/A
Texto (primeiros 100 caracteres): COOP COOPERATIVA DE CUNSURY AGOSTO, 3045 • INHN - Tatuo - . CNPJ57.508.426/0007-63 IE687.134.677.115...
Anotações:
  - tipo: CF
  - cnpj: 57508426000763
  - chave_acesso: 35250157508426000763590004379042087486138245
  - data_emissao: 28/01/2025
  - valor_total_pago: 41.05
  - numero_documento: 208748
  - serie: 
--------------------------------------------------------------------------------


## 4. Preparação dos Dados para Treinamento

Vamos preparar o conjunto de dados para treinar o modelo de reconhecimento de entidades (NER).

In [7]:
def create_annotations_for_ner(texts, extracted_data):
    """
    Cria anotações para treinamento de NER a partir dos textos e dados extraídos.
    
    Args:
        texts: Lista de textos de recibos
        extracted_data: Lista de dicionários com dados extraídos
        
    Returns:
        Lista de dicionários com anotações no formato adequado para NER
    """
    ner_annotations = []
    
    for i, (text, data) in enumerate(zip(texts, extracted_data)):
        # Criar um identificador único para o exemplo
        example_id = f"receipt_{i:04d}"
        
        # Anotar as entidades no texto
        annotations = {}
        for entity_type, entity_value in data.items():
            if entity_value:  # Se o valor não estiver vazio
                annotations[entity_type] = entity_value
        
        # Adicionar ao conjunto de anotações
        ner_annotations.append({
            "id": example_id,
            "text": text,
            "data": annotations
        })
    
    print(f"Criadas {len(ner_annotations)} anotações para NER.")
    return ner_annotations

# Verificar se já temos anotações carregadas
if 'annotations' not in locals() or not annotations:
    print("Criando novas anotações para NER...")
    
    # Verificar se temos textos e dados extraídos disponíveis
    if 'texts' in locals() and texts and 'extracted_data' in locals() and extracted_data:
        if len(texts) == len(extracted_data):
            annotations = create_annotations_for_ner(texts, extracted_data)
            
            # Salvar as anotações para uso futuro
            os.makedirs("dataset_prepared", exist_ok=True)
            with open("dataset_prepared/annotations.json", "w", encoding="utf-8") as f:
                json.dump(annotations, f, ensure_ascii=False, indent=2)
            print(f"Anotações salvas em dataset_prepared/annotations.json")
        else:
            print(f"Erro: número de textos ({len(texts)}) não corresponde ao número de dados extraídos ({len(extracted_data)}).")
    else:
        print("Dados de textos ou extraídos não disponíveis. Verifique as células anteriores.")
else:
    print(f"Já existem {len(annotations)} anotações carregadas. Pulando criação de novas anotações.")

Já existem 27 anotações carregadas. Pulando criação de novas anotações.


## 5. Preparação do Dataset para Treinamento com Transformers

In [8]:
def convert_to_datasets_format(ner_data):
    """
    Converte os dados NER para o formato esperado pela biblioteca Datasets.
    """
    # Dividir em conjuntos de treino, validação e teste
    train_data, test_data = train_test_split(ner_data, test_size=0.2, random_state=SEED)
    train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=SEED)  # 0.25 * 0.8 = 0.2
    
    # Converter para o formato do Datasets
    def convert_format(data):
        return {
            "tokens": [example["tokens"] for example in data],
            "ner_tags": [[label2id[tag] for tag in example["ner_tags"]] for example in data],
            "id": [example["id"] for example in data]
        }
    
    train_dataset = Dataset.from_dict(convert_format(train_data))
    val_dataset = Dataset.from_dict(convert_format(val_data))
    test_dataset = Dataset.from_dict(convert_format(test_data))
    
    # Criar um DatasetDict
    datasets = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    
    return datasets

# Converter para formato de dataset
if 'ner_data' in locals() and ner_data:
    print("Convertendo para formato de dataset...")
    datasets = convert_to_datasets_format(ner_data)
    
    print(f"Criados datasets com:")
    print(f"  - {len(datasets['train'])} exemplos de treino")
    print(f"  - {len(datasets['validation'])} exemplos de validação")
    print(f"  - {len(datasets['test'])} exemplos de teste")
else:
    print("Nenhum dado NER disponível para conversão.")

Nenhum dado NER disponível para conversão.


## 6. Tokenização para o Modelo

In [9]:
def tokenize_and_align_labels(examples):
    """
    Tokeniza os textos e alinha as etiquetas NER aos tokens produzidos pelo tokenizador.
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            # Tokens especiais ([CLS], [SEP], [PAD]) recebem -100 (ignorados na perda)
            if word_idx is None:
                label_ids.append(-100)
            # Para o primeiro token de uma palavra, use a etiqueta correspondente
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Para subpalavras subsequentes, use -100 (ignoradas na perda)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
            
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Carregar o tokenizador
print(f"Carregando tokenizador para o modelo {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenizar e alinhar as etiquetas
if 'datasets' in locals():
    print("Tokenizando datasets...")
    tokenized_datasets = datasets.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=datasets["train"].column_names
    )
    print("Datasets tokenizados e prontos para treinamento.")
else:
    print("Nenhum dataset disponível para tokenização.")

Carregando tokenizador para o modelo neuralmind/bert-base-portuguese-cased...
Nenhum dataset disponível para tokenização.


## 7. Definição do Modelo e Treinamento

In [10]:
# Verificar disponibilidade de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Configuração do modelo
if 'tokenized_datasets' in locals():
    print(f"Carregando modelo base {MODEL_NAME}...")
    model = AutoModelForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(LABELS),
        id2label=id2label,
        label2id=label2id
    )
    model.to(device)
    
    # Configuração de treinamento
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        report_to="none",  # Desabilitar relatório para services externos
    )
    
    # Função de cálculo de métricas
    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)
        
        # Remover tokens ignorados (-100)
        true_predictions = [
            [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        
        # Calcular F1-score
        f1 = f1_score(true_labels, true_predictions)
        return {"f1": f1}
    
    # Colator de dados
    data_collator = DataCollatorForTokenClassification(tokenizer)
    
    # Criar o treinador
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )
    
    print("Modelo configurado e pronto para treinamento.")
    print("Você pode iniciar o treinamento executando a célula abaixo.")
else:
    print("Nenhum dataset tokenizado disponível para treinamento.")

Usando dispositivo: cpu
Nenhum dataset tokenizado disponível para treinamento.


## 8. Treinar o Modelo

Execute esta célula para iniciar o treinamento do modelo. Isso pode levar algum tempo, especialmente se você não tiver uma GPU disponível.

In [11]:
# Treinar o modelo
if 'trainer' in locals():
    print("Iniciando treinamento...")
    trainer.train()
    print("Treinamento concluído!")
    
    # Salvar o modelo treinado
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"Modelo salvo em {OUTPUT_DIR}")
else:
    print("Treinador não está disponível. Por favor, execute as células anteriores primeiro.")

Treinador não está disponível. Por favor, execute as células anteriores primeiro.


## 9. Avaliar o Modelo no Conjunto de Teste

In [12]:
# Avaliar o modelo no conjunto de teste
if 'trainer' in locals() and 'tokenized_datasets' in locals():
    print("Avaliando o modelo no conjunto de teste...")
    test_results = trainer.evaluate(tokenized_datasets["test"])
    print(f"Resultados da avaliação: {test_results}")
    
    # Obter previsões detalhadas
    predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    predictions = np.argmax(predictions, axis=2)
    
    # Remover tokens ignorados (-100)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Imprimir relatório de classificação
    print("Relatório de classificação:")
    print(classification_report(true_labels, true_predictions))
else:
    print("Treinador ou datasets não disponíveis. Por favor, execute as células anteriores primeiro.")

Treinador ou datasets não disponíveis. Por favor, execute as células anteriores primeiro.


## 10. Exemplo de Uso do Modelo Treinado para Extrair Informações de Novos Recibos

In [13]:
def extract_info_from_receipt(text, model, tokenizer):
    """
    Extrai informações de um recibo fiscal usando o modelo treinado.
    
    Args:
        text: Texto do recibo
        model: Modelo NER treinado
        tokenizer: Tokenizador associado ao modelo
        
    Returns:
        Dicionário com as informações extraídas
    """
    # Tokenizar o texto
    tokens = text.split()
    
    # Tokenizar para o modelo
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    )
    
    # Mover para GPU se disponível
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        model = model.to("cuda")
    
    # Obter previsões
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Converter para labels
    predictions = outputs.logits.argmax(dim=2)
    word_ids = inputs.word_ids(batch_index=0)
    
    # Extrair entidades reconhecidas
    entities = {entity: "" for entity in ENTITIES}
    current_entity = None
    current_tokens = []
    
    for idx, (word_idx, pred) in enumerate(zip(word_ids, predictions[0])):
        if word_idx is None:
            continue
            
        label = id2label[pred.item()]
        
        if label.startswith("B-"):
            # Se estava coletando tokens para outra entidade, salvá-los
            if current_entity and current_tokens:
                entities[current_entity] = " ".join(current_tokens)
                
            # Iniciar nova entidade
            current_entity = label[2:]  # Remover o "B-"
            current_tokens = [tokens[word_idx]]
            
        elif label.startswith("I-") and current_entity == label[2:]:
            # Continuar adicionando tokens à entidade atual
            current_tokens.append(tokens[word_idx])
            
        elif label == "O" and current_entity:
            # Finalizar entidade atual
            entities[current_entity] = " ".join(current_tokens)
            current_entity = None
            current_tokens = []
    
    # Lidar com qualquer entidade restante no final
    if current_entity and current_tokens:
        entities[current_entity] = " ".join(current_tokens)
    
    # Retornar as entidades encontradas
    return entities

# Carregar o modelo salvo (se disponível)
try:
    if os.path.exists(OUTPUT_DIR):
        print(f"Carregando modelo salvo de {OUTPUT_DIR}...")
        model = AutoModelForTokenClassification.from_pretrained(OUTPUT_DIR)
        tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
        print("Modelo carregado com sucesso!")
        
        # Exemplo de uso com um texto de teste
        if 'texts' in locals() and texts:
            # Usar um texto do conjunto de dados que não foi usado no treinamento
            test_text = texts[-1]  # Usar o último texto como exemplo
            
            print("\nExtraindo informações do recibo de exemplo...")
            extracted_info = extract_info_from_receipt(test_text, model, tokenizer)
            
            print("\nInformações extraídas:")
            for entity, value in extracted_info.items():
                print(f"{entity}: {value}")
    else:
        print(f"Diretório do modelo {OUTPUT_DIR} não encontrado. Execute o treinamento primeiro.")
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")

Diretório do modelo receipt_ner_model não encontrado. Execute o treinamento primeiro.


## 11. Conclusão

Este notebook demonstrou todo o processo de preparação de dados e treinamento de um modelo de reconhecimento de entidades nomeadas (NER) para extração de informações de recibos fiscais. O fluxo de trabalho incluiu:

1. Extração dos textos do arquivo de dados
2. Preparação e limpeza dos dados
3. Criação de anotações para treinamento do modelo NER
4. Preparação dos datasets para treinamento
5. Treinamento do modelo utilizando um BERT pré-treinado para português
6. Avaliação do modelo no conjunto de teste
7. Uso do modelo para extrair informações de novos recibos

O modelo resultante pode ser usado para automatizar a extração de informações importantes de recibos fiscais, facilitando processos de gestão financeira, contabilidade e conformidade fiscal.